In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [ ]:
import cv2

In [ ]:
!ls ../input

In [ ]:
df=pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')

In [ ]:
dfSample=df.sample(n=300, random_state=1)

In [ ]:
dfMeta=pd.read_csv('../input/gtsrb-german-traffic-sign/Meta.csv')

In [ ]:
dfMerge=pd.merge(dfSample, dfMeta, left_on='ClassId', right_on='ClassId', how='left')

In [ ]:
dfMerge['Path_x'].values

In [ ]:
import cv2
from matplotlib import pyplot as plt


In [ ]:
dfMerge

1. ../input/gtsrb-german-traffic-sign/Train/1/00001_00026_00013.png

In [ ]:
r

In [ ]:
_,seg_img,r=segment('../input/gtsrb-german-traffic-sign/Train/8/00008_00016_00006.png')

In [ ]:
r.shape

In [ ]:
def segment(img_name):
    try:
        
        img=cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=cv2.medianBlur(img,3)
        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        
        dim=(512,512)

        resized = cv2.resize(hsv ,dim, interpolation = cv2.INTER_AREA)
        lower_red = np.array([0,50,50])
        upper_red = np.array([10,255,255])
        mask0 = cv2.inRange(resized, lower_red, upper_red)
        lower_red = np.array([170,50,50])
        upper_red = np.array([180,255,255])
        mask1 = cv2.inRange(resized, lower_red, upper_red)
        mask_red = mask0+mask1

        lower_blue = np.array([110,50,50])
        upper_blue = np.array([130,255,255])
        mask_blue = cv2.inRange(resized, lower_blue, upper_blue)

        mask=mask_blue+mask_red
        kernel = np.ones((7,7),np.uint8)

        # opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        erosion = cv2.erode(mask,kernel,iterations = 3)
        cent , _ = cv2.findContours(erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        areas = [cv2.contourArea(c) for c in cent]
        max_index = np.argmax(areas)
        area=areas[max_index]
        x,y,w,h = cv2.boundingRect(cent[max_index])
        aspect_ratio = float(w)/h
        shape=erosion[y:y+w,x:x+w]
        closing = cv2.morphologyEx(shape, cv2.MORPH_CLOSE, kernel)
        fig=plt.figure(figsize=(8,8))
        fig.add_subplot(4, 1, 1)
        plt.imshow(resized)
        fig.add_subplot(4, 1, 2)
        plt.imshow(erosion)
        fig.add_subplot(4, 1, 3)
        croped=resized[y:y+w,x:x+w]
        plt.imshow(croped)
        img_bolb=np.zeros((512,512))
#         approx = cv2.approxPolyDP(cent[max_index], 0.01*cv2.arcLength(cent[max_index], True), True)
#         CntExternalMask = np.ones(binRed.shape[:2], dtype="uint8") * 255
        cv2.drawContours(img_bolb, cent, contourIdx=max_index, color=(255,255,255),thickness=-1)
        fig.add_subplot(4, 1, 4)
        img_bolb = cv2.morphologyEx(img_bolb, cv2.MORPH_CLOSE, kernel)
        plt.imshow(img_bolb)
        return aspect_ratio,img_bolb,croped
    
    except Exception as e:
        print(e)
        return None,None,None

In [ ]:
def rgb2hsi (rgb_img):
    row=np.shape (rgb_img) [0]
    col=np.shape (rgb_img) [1]
    hsi_img=rgb_img.copy ()
    #Channel split the image
    b, g, r=cv2.split (rgb_img)
    #Normalize the channel to [0,1]
    [b, g, r]=[i/255.0 for i in ([b, g, r])]
    h=np.zeros ((row, col)) #define h channel
    i=(r + g + b)/3.0 #Calculate the i channel
    s=np.zeros ((row, col)) #define s channel
    for i in range (row):
        den=np.sqrt ((r [i] -g [i]) ** 2+ (r [i] -b [i]) * (g [i] -b [i]))
        thetha=np.arccos (0.5 * (r [i] -b [i] + r [i] -g [i])/den) #Calculate the angle
        h=np.zeros (col) #define temporary array
        #den>0 and g>= b element h is assigned thetha
        h [b [i]<= g [i]]=thetha [b [i]<= g [i]]
        #den>0 and the element h of g<= b is assigned thetha
        h [g [i]<b [i]]=2 * np.pi-thetha [g [i]<b [i]]
        #den<0's element h is assigned a value of 0
        h [den == 0]=0
        h [i]=h/(2 * np.pi) #Assign to h channel after radian
        #Calculate s channel
        for i in range (row):
            min=[]
            #Find the minimum value of each group of rgb values
            for j in range (col):
                arr=[b [i] [j], g [i] [j], r [i] [j]]
                min.append (np.min (arr))
            min=np.array (min)
            #Calculate s channel
            s [i]=1-min * 3/(r [i] + b [i] + g [i])
            #i is a value of 0 directly assigned 0
            s [i] [r [i] + b [i] + g [i] == 0]=0
            #Expand to 255 for easy display,Generally h component is between [0,2pi] and s and i are between [0,1]
            hsi_img [:,:, 0]=h * 255
            hsi_img [:,:, 1]=s * 255
            hsi_img [:,:, 2]=i * 255
    return hsi_img

In [ ]:
def RGB_TO_HSI(img):

    with np.errstate(divide='ignore', invalid='ignore'):

        #Load image with 32 bit floats as variable type
        bgr = np.float32(img)/255

        #Separate color channels
        blue = bgr[:,:,0]
        green = bgr[:,:,1]
        red = bgr[:,:,2]

        #Calculate Intensity
        def calc_intensity(red, blue, green):
            return np.divide(blue + green + red, 3)

        #Calculate Saturation
        def calc_saturation(red, blue, green):
            minimum = np.minimum(np.minimum(red, green), blue)
            saturation = 1 - (3 / (red + green + blue + 0.001) * minimum)

            return saturation

        #Calculate Hue
        def calc_hue(red, blue, green):
            hue = np.copy(red)

            for i in range(0, blue.shape[0]):
                for j in range(0, blue.shape[1]):
                    hue[i][j] = 0.5 * ((red[i][j] - green[i][j]) + (red[i][j] - blue[i][j])) / \
                                math.sqrt((red[i][j] - green[i][j])**2 +
                                        ((red[i][j] - blue[i][j]) * (green[i][j] - blue[i][j])))
                    hue[i][j] = math.acos(hue[i][j])

                    if blue[i][j] <= green[i][j]:
                        hue[i][j] = hue[i][j]
                    else:
                        hue[i][j] = ((360 * math.pi) / 180.0) - hue[i][j]

            return hue

        #Merge channels into picture and return image
        hsi = cv2.merge((calc_hue(red, blue, green), calc_saturation(red, blue, green), calc_intensity(red, blue, green)))
        return hsi

In [ ]:
seg_img.shape

In [ ]:
d1,d2

In [ ]:
plt.imshow(seg_img)

In [ ]:
circle='../input/gtsrb-german-traffic-sign/Meta/0.png'
triangle='../input/gtsrb-german-traffic-sign/Meta/11.png'
rectangle='../input/gtsrb-german-traffic-sign/Meta/12.png'

In [ ]:
_,circle_segmented,r=segment(circle)

In [ ]:
circle='../input/gtsrb-german-traffic-sign/Meta/0.png'
triangle='../input/gtsrb-german-traffic-sign/Meta/11.png'
rectangle='../input/gtsrb-german-traffic-sign/Meta/12.png'
_,circle_segmented,r=segment(circle)
_,triangle_segmented,r=segment(triangle)
d1 = cv2.matchShapes(seg_img,circle_segmented,cv2.CONTOURS_MATCH_I2,0)
d2 = cv2.matchShapes(seg_img,triangle_segmented,cv2.CONTOURS_MATCH_I2,0)

In [ ]:
d1,d2

In [ ]:
from skimage.feature import hog
from skimage import data, exposure

In [ ]:
fd, hog_image = hog(r, orientations=8, pixels_per_cell=(5, 5),cells_per_block=(2, 2), visualize=True, multichannel=True)

In [ ]:
fd.shape

In [ ]:
import math

In [ ]:
plt.imshow(resized)

In [ ]:
img=cv2.imread('../input/gtsrb-german-traffic-sign/Meta/12.png')

In [ ]:
plt.imshow(img)

p

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(resized)


In [ ]:
kernel = np.ones((7,7),np.uint8)

# opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
erosion = cv2.erode(mask,kernel,iterations = 3)

In [ ]:
plt.imshow(erosion)


In [ ]:
rect=cv2.rectangle(erosion,(x,y),(x+w,y+h),(255,255,0),3)

In [ ]:
plt.imshow(erosion)

In [ ]:
aspect_ratio = float(w)/h

In [ ]:
aspect_ratio

In [ ]:
extent = float(area)/w*h

In [ ]:
(w*h)/3,(w*h)/25,extent

In [ ]:
shape=erosion[y:y+h,x:x+w]
closing = cv2.morphologyEx(shape, cv2.MORPH_CLOSE, kernel)

In [ ]:
plt.imshow(closing)

In [ ]:
moments = cv2.moments(closing)
huMoments = cv2.HuMoments(moments)

In [ ]:
d2 = cv2.matchShapes(im1,im2,cv2.CONTOURS_MATCH_I2,0)